In [ ]:
%matplotlib inline
from astropy.table import Table
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

mpl.rc('text', usetex=True)

In [ ]:
def poisson_upper(n, S):
    return n + S*np.sqrt(n + 0.75) + (S**2 + 3.)/4.
def poisson_lower(n, S):
    return n - S*np.sqrt(n - 0.25) + (S**2 - 1.)/4

def calc_1sigma_poisson_bounds(n, ntot):
    upper = poisson_upper(n, 1.0)
    lower = poisson_lower(n, 1.0)
    return [(n-lower)/ntot, (upper-n)/ntot]

# Load the classified catalog

In [ ]:
infall = Table().read('../catalogs/classified_surfacedensity.csv')
infall = infall[np.where((infall['morph'] != 'C') &
                         (infall['morph'] != 'U'))]
infall.sort('surface_density_compcorred')

# Load the ediscs cluster core data

In [ ]:
cores = Table().read('../catalogs/classified_surfacedensity_desai.csv')
cores.sort('surface_density_compcorred')

# Load the High-z results from Postman+2005

In [ ]:
def add_LL(table, col):
    sigma = table[col+'_UL'] - table[col]
    table[col+'_LL'] = table[col] - sigma
    table[col+'_LL'][np.where(table[col+'_LL'] < 0.0)] = 0.0
    table[col+'_UL'][np.where(table[col+'_UL'] > 1.0)] = 1.0
    table[col+'_extent'] = table[col+'_UL'] - table[col+'_LL']
    return table

In [ ]:
postman = Table(np.loadtxt('./external-data/postman05_fractions.dat', skiprows=1),
                names=('rho', 'fE', 'fE_UL', 'fS0', 'fS0_UL', 'fearly', 'fearly_UL', 'flate', 'flate_UL'))

# Add lower limits to the table
for col in ['fE', 'fS0', 'fearly', 'flate']:
    postman = add_LL(postman, col)

# Load the low-z results from Dresler 1980

In [ ]:
def add_dressler_LL(table, col):
    sigma = table[col+'_e']
    table[col+'_LL'] = table[col] - sigma
    table[col+'_LL'][np.where(table[col+'_LL'] < 0.0)] = 0.0
    table[col+'_extent'] = 2.0*table[col+'_e']

    for row in table:
        if row[col+'_LL'] + row[col+'_extent'] > 1.0:
            row[col+'_extent'] = 1.0 - row[col+'_LL']
    
    return table

In [ ]:
cosmologycorrection = (223.91**2)/(163.23**2)

dressler = Table(np.loadtxt('./external-data/dressler80_fullResults_withErrors.dat', skiprows=1),
                 names=('logrho', 'fE', 'fE_e', 'fS0', 'fS0_e', 'flate', 'flate_e'))
dressler['fearly'] = dressler['fE'] + dressler['fS0']
dressler['fearly_e'] = np.sqrt(dressler['fE_e']**2 + dressler['fS0_e']**2)
dressler['rho'] = 10.**dressler['logrho'] * cosmologycorrection

for col in ['fE', 'fS0', 'fearly', 'flate']:
    dressler = add_dressler_LL(dressler, col)

# Make even bins in surface density for the infall data

In [ ]:
n_bins_infall = 5

n_per_bin = int(len(infall)/n_bins_infall)
infall['rhobin'] = np.ones(len(infall), dtype='int8')*-1
counter, setval = 0, 0

for row in infall:
    row['rhobin'] = setval
    counter += 1
    if int(counter) == int(n_per_bin):
        counter = 0
        setval += 1

# We have 1 straggler, put it in the last bin
infall[-1]['rhobin'] = 4

# Make even bins in surface density for the core data

In [ ]:
n_bins_core = 10

n_per_bin = int(len(cores)/n_bins_core)
cores['rhobin'] = -1*np.ones(len(cores), dtype='int8')
counter, setval = 0, 0

for row in cores:
    row['rhobin'] = setval
    counter += 1
    if int(counter) == int(n_per_bin):
        counter = 0
        setval += 1

# Plot the fractions in each bin

In [ ]:
def compute_binned_fractions(table, morph):
    binids = list(set(table['rhobin']))
    ntot, nmorph = np.ones(len(binids))*-1, np.ones(len(binids))*-1
    rhomin, rhomax = np.ones(len(binids))*-1, np.ones(len(binids))*-1
    rhomean = np.ones(len(binids))*-1
    for i in binids:
        onebin = table[np.where(table['rhobin'] == i)]
        ntot[i] = float(len(onebin))
        nmorph[i] = float(len(onebin[np.where(onebin['morph'] == morph)]))
        rhomin[i] = min(onebin['surface_density_compcorred'])
        rhomax[i] = max(onebin['surface_density_compcorred'])
        rhomean[i] = np.average(onebin['surface_density_compcorred'])
    return nmorph, {'nmorph': nmorph, 'ntot': ntot, 'rhomin': rhomin, 'rhomax': rhomax, 'rhomean': rhomean}

#### Subplot details

In [ ]:
plotdeetz = [
    {
        'morph': ["E"], 
        'ylabel': r'$f_{\rm E}$',
        'oldcol': 'fE'
    },
    {
        'morph': ["S0"], 
        'ylabel': r'$f_{\rm S0}$',
        'oldcol': 'fS0'
    },
    {
        'morph': ["E", "S0"], 
        'ylabel': r'$f_{\rm E+S0}$',
        'oldcol': 'fearly'
    },
    {
        'morph': ["Irr", "Sa", "Sb", "Sc", "Sd", "Sm"], 
        'ylabel': r'$f_{\rm Sp+Irr}$',
        'oldcol': 'flate'
    }
]

#### Make the plot

In [ ]:
ebarkwargs = {'ms':2, 'elinewidth':0.5, 'capthick':0.5, 'capsize':2}
postmanargs = {'facecolors': '#FF8080', 'edgecolors': '#FF0000', 'alpha': 0.3, 'hatch': 'x'}
dresslerargs = {'facecolors': '#D1E3FF', 'edgecolors': '#5282CC', 'alpha': 0.3}

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(6,4), dpi=250)
axs = axs.flatten()
xlabel = r'$\rho_{\rm proj}$'

for ax, opts in zip(axs, plotdeetz):
    # Compute the fraction in infall region
    nmorphs_infall = np.zeros(n_bins_infall)
    for morph in opts['morph']:
        nmorph_single, res_infall = compute_binned_fractions(infall, morph)
        nmorphs_infall+=nmorph_single

    # Compute the fraction in cores
    nmorphs_core = np.zeros(n_bins_core)
    for morph in opts['morph']:
        nmorph_single, res_core = compute_binned_fractions(cores, morph)
        nmorphs_core+=nmorph_single        
    
    # Plot the infall clusters by morphology
    ax.errorbar(res_infall['rhomean'], nmorphs_infall/res_infall['ntot'], 
                xerr=[res_infall['rhomean'] - res_infall['rhomin'], res_infall['rhomax'] - res_infall['rhomean']],
                yerr=calc_1sigma_poisson_bounds(nmorphs_infall, res_infall['ntot']),
                fmt='ok', **ebarkwargs,)
    
    # Plot the cluster centers data
    ax.errorbar(res_core['rhomean'], nmorphs_core/res_core['ntot'], 
                xerr=[res_core['rhomean'] - res_core['rhomin'], res_core['rhomax'] - res_core['rhomean']],
                yerr=calc_1sigma_poisson_bounds(nmorphs_core, res_core['ntot']),
                fmt='^g', **ebarkwargs,)
    
    # Plot the postman results
    for row in postman:
        ax.broken_barh(xranges=[[row['rho']*10**(-0.15), row['rho']*(10**0.15 - 10**(-0.15))]],
                       yrange=[row[opts['oldcol']+'_LL'], row[opts['oldcol']+'_extent']], **postmanargs)
    
    # Plot the Dressler results
    for row in dressler:
        ax.broken_barh(xranges=[[row['rho']*10**(-0.1), row['rho']*(10**0.1 - 10**(-0.1))]],
                       yrange=[row[opts['oldcol']+'_LL'], row[opts['oldcol']+'_extent']], **dresslerargs)
    
    
    # Format the axes
    ax.set_xscale('log')
    ax.set_xlim((1, 2000))
    ax.set_ylim((-0.1, 1.1))
    ax.set_yticks((0.0, .2, .4, .6, .8, 1.))
    ax.set_ylabel(opts['ylabel'])
    ax.tick_params(which='both', direction='in', top=True, right=True, left=True, bottom=True)

# General axis formatting
yticks = (0.0, .2, .4, .6, .8, 1.)

axs[0].set_xticklabels(());

axs[1].set_xticklabels(());
axs[1].yaxis.tick_right()
axs[1].yaxis.set_label_position('right')

axs[2].set_xlabel(xlabel)

axs[3].set_xlabel(xlabel)
axs[3].yaxis.tick_right()
axs[3].yaxis.set_label_position('right')

fig.subplots_adjust(wspace=.001, hspace=.001)
plt.savefig('plots/morphology-density-relation.pdf', bbox_inches='tight')